<a href="https://colab.research.google.com/github/2025-01-sookmyung-opensource/final_project/blob/main/Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. 필요한 패키지 설치
!pip install -q streamlit pyngrok ultralytics gdown opencv-python pillow

In [ ]:
# 2. pyngrok 임포트 및 인증 토큰 설정
from pyngrok import ngrok

# ngrok 인증 토큰 마스킹
ngrok.set_auth_token("2p")

# 3. 기존에 열려있는 ngrok 세션 종료 (중복 방지)
!pkill -f ngrok

In [ ]:
# 4. Google Drive에서 모델 다운로드
import gdown
gdown.download(id='1sl1dXyWuJiXjhHeacomWwfppsM57VJ85', output='best.pt', quiet=False)

# 5. Streamlit 앱 저장
app_code = r"""
import streamlit as st
from PIL import Image
import numpy as np
import cv2
from ultralytics import YOLO

# ─────────────────────────  설정  ─────────────────────────
st.set_page_config(page_title='🍱 식단 이미지 분석기', layout='centered')
st.title('🍱 식단 이미지 분석기')
st.markdown('업로드한 음식 사진에서 YOLOv8로 음식을 탐지하고, '
            '각 객체를 잘라 다시 분석합니다.')

# 모델 로드
@st.cache_resource
def load_model():
    return YOLO('best.pt')
model = load_model()

# ─────────────────────────  업로드  ─────────────────────────
uploaded = st.file_uploader('이미지를 업로드하세요', ['jpg', 'jpeg', 'png'])

if uploaded:
    pil_img = Image.open(uploaded).convert('RGB')
    rgb = np.array(pil_img)                         # (H,W,RGB) for display
    bgr = cv2.cvtColor(rgb, cv2.COLOR_RGB2BGR)      # (H,W,BGR) for YOLO

    st.image(rgb, caption='업로드한 이미지', use_column_width=True)

    # ─────────────  1차 예측  ─────────────
    res = model.predict(bgr, verbose=False)[0]
    names = model.names
    cls_ids = res.boxes.cls.cpu().numpy().astype(int)
    boxes   = res.boxes.xyxy.cpu().numpy()

    # 클래스별 개수 집계
    detected = {}
    for cid in cls_ids:
        name = names[cid]
        detected[name] = detected.get(name, 0) + 1

    st.subheader('🍽️ 인식된 항목')
    if detected:
        for k, v in detected.items():
            st.write(f'- {k}: {v}개')
    else:
        st.write('아무것도 인식되지 않았습니다.')

    # 전체 시각화 (result.plot() 는 이미 RGB ➜ 그대로 사용)
    st.image(res.plot(), caption='YOLO 전체 인식 결과', use_column_width=True)

    # ─────────────  개별 객체 + 2차 예측  ─────────────
    st.subheader('🔍 인식된 객체 및 상세 분석')
    for i, (cid, box) in enumerate(zip(cls_ids, boxes), start=1):
        x1, y1, x2, y2 = map(int, box)
        crop_rgb = rgb[y1:y2, x1:x2]                # 표시용 (RGB)
        crop_bgr = cv2.cvtColor(crop_rgb, cv2.COLOR_RGB2BGR)  # 추론용

        if crop_rgb.size == 0:
            continue

        st.image(crop_rgb, caption=f'{i}. {names[cid]}', width=240)

        # 2차 예측
        sub = model.predict(crop_bgr, verbose=False)[0]
        sub_ids = sub.boxes.cls.cpu().numpy().astype(int)
        sub_det = {}
        for sid in sub_ids:
            sname = names[sid]
            sub_det[sname] = sub_det.get(sname, 0) + 1

        if sub_det:
            st.markdown(f'**{i}. {names[cid]} 내부 분석 결과**')
            for sname, scnt in sub_det.items():
                st.write(f'↳ {sname}: {scnt}개')

        st.image(sub.plot(), caption=f'{i}. 상세 분석 시각화', width=300)
"""

with open('app.py', 'w') as f:
    f.write(app_code)

# 6. Streamlit 백그라운드 실행
!nohup streamlit run app.py > /dev/null 2>&1 &

# 7. 외부 접속 주소 생성
public_url = ngrok.connect(8501)
print(f'✅ 앱이 준비되었습니다: {public_url}')

Downloading...
From: https://drive.google.com/uc?id=1sl1dXyWuJiXjhHeacomWwfppsM57VJ85
To: /content/best.pt
100%|██████████| 6.26M/6.26M [00:00<00:00, 40.7MB/s]


✅ 접속 주소: NgrokTunnel: "https://d5f2-34-16-160-81.ngrok-free.app" -> "http://localhost:8501"


